In [ ]:
!export CUDA_VISIBLE_DEVICES='2'

In [ ]:
import pickle
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import torch
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
## combining text features to one
TEXT_FEATURES_PATH = 'features_text_instructions'
features = torch.tensor([])
for i in sorted(os.listdir(TEXT_FEATURES_PATH), key=lambda x: int(x.split('.')[0])):
    if 'pkl' in i:
        print(i)
        load_file = torch.load(os.path.join(TEXT_FEATURES_PATH, i))
        if features.size(0) == 0:
            features = load_file
        else:
            features = torch.vstack([features, load_file])
features = features.cpu()
torch.save(features, 'instructions_text_best.pkl')

In [ ]:
IMAGE_FEATURES_PATH = 'gen_data_val'
features = torch.tensor([])
for i in sorted(os.listdir(IMAGE_FEATURES_PATH)):
    pre_pre_path = os.path.join(IMAGE_FEATURES_PATH, i)
    print(pre_pre_path)
    for j in sorted(os.listdir(pre_pre_path)):
        pre_path = os.path.join(pre_pre_path, j)
        print(pre_path)
        all_files = sorted(os.listdir(pre_path))
        image_features = sorted(all_files[:-1], key=lambda x: int(x))
        paths = all_files[-1]
        for k in image_features:
            feature = torch.load(os.path.join(pre_path, k))
            if features.size(0) == 0:
                features = feature
            else:
                features = torch.vstack([features, feature])
    print(features.size())
torch.save(features.cpu(), 'full_image_val.pkl')

In [ ]:
TEXT_LAYERS = 'recipe1M_layers/layer2.json'
f = open(TEXT_LAYERS, 'rb')
data_map = json.load(f)

In [ ]:
data_map_list = []
for i in data_map:
    for j in i['images']:
        data_map_list.append([i['id'], j['id']])

In [ ]:
data_map_df = pd.DataFrame(data_map_list, columns=['id', 'image_id'])
data_map_df['id'] = data_map_df['id'].astype(str)

In [ ]:
data_map_df.reset_index(inplace=True)
data_map_df.drop(columns=['index'], inplace=True)
data_map_df = data_map_df.set_index('id')
data_map_df.head()

In [ ]:
TEXT_LAYERS_1 = 'recipe1M_layers/layer1.json'
f = open(TEXT_LAYERS_1, 'rb')
text_map = json.load(f)

In [ ]:
text_data_list = []
for i in text_map:
    text_data_list.append(i['id'])

In [ ]:
text_map_df = pd.DataFrame(text_data_list, columns=['id'])
text_map_df['id'] = text_map_df['id'].astype(str)
text_map_df.reset_index(inplace=True)
# text_map_df.drop(columns=['index'], inplace=True)
text_map_df = text_map_df.set_index('id')
text_map_df.head()

In [ ]:
test = text_map_df.join(data_map_df)

In [ ]:
test.reset_index(inplace=True)

In [ ]:
import numpy as np
test.fillna(0, inplace=True)

In [ ]:
text_with_images_map_df = test[test['image_id']!=0]

In [ ]:
IMAGE_FEATURES_PATH = 'gen_data_train'
image_paths = []
for i in sorted(os.listdir(IMAGE_FEATURES_PATH)):
    pre_pre_path = os.path.join(IMAGE_FEATURES_PATH, i)
    for j in sorted(os.listdir(pre_pre_path)):
        pre_path = os.path.join(pre_pre_path, j)
        all_files = sorted(os.listdir(pre_path))
        paths = all_files[-1]
        f = open(os.path.join(pre_path, paths), 'r')
        img_paths = f.readlines()
        for k in img_paths:
            image_paths.append([k.strip(),k.strip().split('/')[-1]])

In [ ]:
image_map_df = pd.DataFrame(image_paths, columns=['path', 'image_id'])

In [ ]:
image_map_df.reset_index(inplace=True)
image_map_df = image_map_df.set_index('image_id')
image_map_df.head()

In [ ]:
text_with_images_map_df.reset_index(inplace=True)
text_with_images_map_df = text_with_images_map_df.set_index('image_id')
full_data_map = text_with_images_map_df.join(image_map_df, lsuffix='_text', rsuffix='_image')

In [ ]:
train_map = full_data_map.fillna(-1)
train_map = train_map[train_map['index_image'] != -1]

In [ ]:
train_map.reset_index(inplace=True)
train_map['index_image'] = train_map['index_image'].astype(int)
train_map.head(10)

In [ ]:
IMAGE_FEATURES_PATH = 'gen_data_train'
counter = []
for i in sorted(os.listdir(IMAGE_FEATURES_PATH)):
    pre_pre_path = os.path.join(IMAGE_FEATURES_PATH, i)
    for j in sorted(os.listdir(pre_pre_path)):
        pre_path = os.path.join(pre_pre_path, j)
        all_files = sorted(os.listdir(pre_path))
        paths = all_files[-1]
        f = open(os.path.join(pre_path, paths), 'r')
        img_paths = f.readlines()
        image_features = sorted(all_files[:-1], key=lambda x: int(x))
        img_counter = 0
        for k in image_features:
            feature = torch.load(os.path.join(pre_path, k))
            img_counter += feature.size(0)
        if len(img_paths) != img_counter:
            print('here', os.path.join(pre_path, paths))
        counter.append((len(img_paths), img_counter))

In [ ]:
img_features = torch.load('full_image_complete.pkl')

In [ ]:
text_features = torch.load('full_text_best.pkl')

In [ ]:
train_text_features = text_features[train_map['index_text']]
train_img_features = img_features[train_map['index_image']]

In [ ]:
torch.save(train_img_features, 'full_img_train.pkl')
torch.save(train_text_features, 'full_text_best_train.pkl')